In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense,Activation,Attention,Bidirectional,Concatenate
from tensorflow.keras.models import load_model
from difflib import SequenceMatcher
import pickle
#import encoding

INPUT_LENGTH = 90
OUTPUT_LENGTH = 90

In [2]:
##### Limit GPU for training ###
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        pass
        #print(e)

In [3]:
file = open('char2int.pickle', 'rb')
# dump information to that file
char2int = pickle.load(file)
file2 = open('int2char.pickle', 'rb')
# dump information to that file
int2char= pickle.load(file2)

In [4]:
model = load_model('model_lstm_with_attention3.h5',compile=False)

In [17]:
def generate(text):
    encoder_input = encoding.transform(char2int, [text], 90)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = encoding.CHAR_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return decoder_input[:,1:]

def decode(decoding, sequence):
    text = ''
    text_list =[]
    for i in sequence:
        if i == 0:
            break
        text += int2char[i]
        text_list.append(int2char[i])
    return text,text_list

def correct_text(text):
    decoder_output = generate(text)
    txt,lst = decode(int2char, decoder_output[0])
    text_result = txt.strip('\t')
    return text_result,lst[1:]

In [28]:
text = 'โรงพยาบาลชัยภูมิ'

In [24]:
import time

In [29]:
start_time = time.time()
a,b=correct_text(text)
print("--- %s seconds ---" % (time.time() - start_time))

--- 8.445616960525513 seconds ---


In [30]:
a

'โรงพยาบาลชัยภูมิ'